In [1]:
import datetime
import json
import logging
from datetime import timedelta

import alaska_utils
import dask
import eqt_utils
import matplotlib.pyplot as plt
import numpy as np
import obspy
import pandas as pd
import postprocess
import requests
import seisbench
import seisbench.models as sbm

# Progress bar for dask operations:
from dask.diagnostics import ProgressBar
from obspy.clients.fdsn import Client

pbar = dask.diagnostics.ProgressBar()
pbar.register()

import warnings

warnings.filterwarnings("ignore")

### This notebook allows you to apply EQTransformer (by way of Seisbench) to continuous seismic data from the AACSE network, and retrieve picks for the specified time period. The waveforms can also be filtered or denoised by DeepDenoiser (by way of Seisbench).

#### Choose window length (s), window overlap (s), and start and end times

In [2]:
waveform_length = 30
waveform_overlap = 0
starttime = datetime.datetime(2019, 5, 27, 10, 30)
endtime = datetime.datetime(2019, 5, 27, 11, 30)

#### Load list of stations to run the workflow on, specify by network and maximum station elevation if desired

In [3]:
# Pre=saved station list in pandas dataframe format
dfS = pd.read_parquet(
    "https://github.com/zoekrauss/alaska_catalog/raw/main/data_acquisition/alaska_stations.parquet"
)
dfS = alaska_utils.station_list(dfS,starttime,endtime,elevation=False,network=False)

#### Specify whether to apply any filtering to the waveforms before phase picking

This choice is made with the variable filt_type.

filt_type = 0 for raw waveforms\
filt_type = 1 for bandpass filtered waveforms\
filt_type = 2 to apply DeepDenoiser to the waveforms

NOTE: if specifying bandpass filter, must also specify f1 and f2 for bandpass limits

In [4]:
filt_type = 2
f1 = 5
f2 = 35

### Run the workflow, and output:
#### (1) pandas dataframe of all pick metadata
#### (2) list of pick dictionaries formatted for use with GAMMA

In [ ]:
%%time

pick_info,gamma_picks = eqt_utils.ml_pick(
    dfS, starttime, endtime, waveform_length, waveform_overlap, filt_type, f1=f1, f2=f2
)

[########################################] | 100% Completed | 53.5s
Status: denoising data using DeepDenoiser


2022-03-15 16:51:04,642 | seisbench | WARNING | Parts of the input stream consist of fragments shorter than the number of input samples. Output might be empty.
2022-03-15 16:51:04,643 | seisbench | WARNING | Parts of the input stream consist of fragments shorter than the number of input samples. Output might be empty.
2022-03-15 16:51:04,643 | seisbench | WARNING | Parts of the input stream consist of fragments shorter than the number of input samples. Output might be empty.
2022-03-15 16:51:04,658 | seisbench | WARNING | Parts of the input stream consist of fragments shorter than the number of input samples. Output might be empty.
2022-03-15 16:51:04,658 | seisbench | WARNING | Parts of the input stream consist of fragments shorter than the number of input samples. Output might be empty.
2022-03-15 16:51:04,659 | seisbench | WARNING | Parts of the input stream consist of fragments shorter than the number of input samples. Output might be empty.
2022-03-15 16:51:04,783 | seisbench | WA

Status: applying EQTransformer


2022-03-15 16:51:53,944 | seisbench | WARNING | Parts of the input stream consist of fragments shorter than the number of input samples. Output might be empty.
2022-03-15 16:51:53,948 | seisbench | WARNING | Parts of the input stream consist of fragments shorter than the number of input samples. Output might be empty.
2022-03-15 16:51:53,953 | seisbench | WARNING | Parts of the input stream consist of fragments shorter than the number of input samples. Output might be empty.
2022-03-15 16:51:53,957 | seisbench | WARNING | Parts of the input stream consist of fragments shorter than the number of input samples. Output might be empty.
2022-03-15 16:51:53,961 | seisbench | WARNING | Parts of the input stream consist of fragments shorter than the number of input samples. Output might be empty.
2022-03-15 16:51:53,965 | seisbench | WARNING | Parts of the input stream consist of fragments shorter than the number of input samples. Output might be empty.
2022-03-15 16:51:53,969 | seisbench | WA

#### Below is an example of the output pick dataframe. You can see that both the raw and denoised SNR of the pick is saved for all channels, as well as the raw and denoised amplitude of the signal (for use in magnitude estimation with GAMMA), and the probability of the pick from EQTransformer.

In [14]:
pick_info.head()

,id,network,station,channel,phase,timestamp,prob,raw_amp,den_amp,z_raw_snr,z_den_snr,n_raw_snr,n_den_snr,e_raw_snr,e_den_snr
0,XO.LT01..HH,XO,LT01,HH,P,2019-05-27T10:30:37.752,0.838696,1.205701e+06,1.205701e+06,76.891323,76.891323,29.990748,29.990748,3.457951,3.457951
1,XO.LT01..HH,XO,LT01,HH,S,2019-05-27T10:30:39.252,0.381929,2.580351e+03,2.580351e+03,0.030619,0.030619,25.036566,25.036566,3.282618,3.282618
2,XO.WD50..HH,XO,WD50,HH,P,2019-05-27T10:30:05.249,0.351032,2.296432e+02,2.296432e+02,1.780624,1.780624,1.244334,1.244334,2.107208,2.107208
3,XO.WD50..HH,XO,WD50,HH,S,2019-05-27T10:30:16.599,0.494853,2.016174e+03,2.016174e+03,8.466265,8.466265,29.058924,29.058924,17.042699,17.042699
4,XO.WD51..HH,XO,WD51,HH,P,2019-05-27T10:30:05.266,0.765708,4.815866e+02,4.815866e+02,6.566353,6.566353,15.821010,15.821010,18.003864,18.003864


### Save to parquet

In [15]:
pick_info.to_parquet("picks_20190527_1030_bp0535.parquet", version="2.6")